In [18]:
!pip install fastai

## Training with fast.ai

LM fine-tuning ⇒ sentiment analysis 수행하는 텍스트 분류 모델 ⇒ ULMFiT

fast.ai(딥러닝 라이브러리)에 내장되어있는 IMDb 영화 리뷰 데이터

텍스트 샘플은 이미 Tokenization이 진행되어 xxmaj 같은 이상한 토큰들이 들어가있음

트랜스포머 대신 AWD_LSTM 아키텍처로 (사전훈련된) 텍스트 분류기 생성

이 분류기는 위키피디아 문서를 기반으로 pretrained, fine-tuning까지 마친 상태

이걸 바로 쓰기보단 IMDb 데이터셋에 맞춰 모델을 fine-tuning 한 후 학습할 예정 (ULMFiT)

ULMFiT이란?


- pretrained LLM을 가져옴. 일반적인 영어 문법 구조 학습한 상태
- label 없는 데이터로 fine-tuning. 다음 단어 예측으로 해당 도메인 데이터의 텍스트 스타일과 용어에 적응
- label 있는 데이터로 최종 분류 모델 학습. 이전에 학습한 도메인 지식을 활용하여 높은 성능 달성할 수 있음.

In [ ]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [ ]:
dls = TextDataLoaders.from_folder(path, valid='test') # 내장 데이터셋 다운로드

2025-03-12 17:24:21.584650: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
dls.show_batch()

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * * attention xxmaj spoilers * * \n\n xxmaj first of all , let me say that xxmaj rob xxmaj roy is one of the best films of the 90 's . xxmaj it was an amazing achievement for all those involved , especially the acting of xxmaj liam xxmaj neeson , xxmaj jessica xxmaj lange , xxmaj john xxmaj hurt , xxmaj brian xxmaj cox , and xxmaj tim xxmaj roth . xxmaj michael xxmaj canton xxmaj jones painted a wonderful portrait of the honor and dishonor that men can represent in themselves . xxmaj but alas … \n\n it constantly , and unfairly gets compared to "" braveheart "" . xxmaj these are two entirely different films , probably only similar in the fact that they are both about xxmaj scots in historical xxmaj scotland . xxmaj yet , this comparison frequently bothers me because it seems",pos
2,"xxbos * ! ! - xxup spoilers - ! ! * \n\n xxmaj before i begin this , let me say that i have had both the advantages of seeing this movie on the big screen and of having seen the "" authorized xxmaj version "" of this movie , remade by xxmaj stephen xxmaj king , himself , in 1997 . \n\n xxmaj both advantages made me appreciate this version of "" the xxmaj shining , "" all the more . \n\n xxmaj also , let me say that xxmaj i 've read xxmaj mr . xxmaj king 's book , "" the xxmaj shining "" on many occasions over the years , and while i love the book and am a huge fan of his work , xxmaj stanley xxmaj kubrick 's retelling of this story is far more compelling … and xxup scary . \n\n xxmaj kubrick",pos
3,"xxbos i thought that xxup rotj was clearly the best out of the three xxmaj star xxmaj wars movies . i find it surprising that xxup rotj is considered the weakest installment in the xxmaj trilogy by many who have voted . xxmaj to me it seemed like xxup rotj was the best because it had the most profound plot , the most suspense , surprises , most xxunk the ending ) and definitely the most episodic movie . i personally like the xxmaj empire xxmaj strikes xxmaj back a lot also but i think it is slightly less good than than xxup rotj since it was slower - moving , was not as episodic , and i just did not feel as much suspense or emotion as i did with the third movie . \n\n xxmaj it also seems like to me that after reading these surprising reviews that",pos
4,"xxbos xxmaj okay , so xxmaj i 'm not a big video game buff , but was the game xxmaj house of the xxmaj dead really famous enough to make a movie from ? xxmaj sure , they went as far as to actually put in quick video game clips throughout the movie , as though justifying any particular scene of violence , but there are dozens and dozens of games that look exactly the same , with the hand in the bottom on the screen , supposedly your own , holding whatever weapon and goo - ing all kinds of aliens or walking dead or snipers or whatever the case may be . \n\n xxmaj it 's an interesting premise in xxmaj house of the xxmaj dead , with a lot of college kids ( loaded college kids , as it were , kids who are able to pay",neg
5,"xxbos xxmaj here are the matches . . . ( adv . = advantage ) \n\n xxmaj the xxmaj warriors ( ultimate xxmaj warrior , xxmaj texas xxmaj tornado and xxmaj legi

In [ ]:
# text classification model 생성
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
# 학습이 복잡한 tfm 대신 AWD_LSTM 사용

In [5]:
learn

In [ ]:
learn.fine_tune(4, 1e-2) # epoch 4, lr 1e-2
# 자동적으로 ULMFiT 기법 적용되는 함수
# fine_tune 함수를 한번 실행하면 내부적으로 최상위 레이어 학습 -> 전체 모델 학습 과정이 이루어짐

epoch,train_loss,valid_loss,accuracy,time
0,0.456661,0.401192,0.819560,00:44


epoch,train_loss,valid_loss,accuracy,time
0,0.318195,0.239310,0.904360,01:19
1,0.242654,0.201179,0.919760,01:20
2,0.182149,0.192862,0.926760,01:19
3,0.146465,0.203134,0.927360,01:20


In [7]:
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.147833,0.197472,0.927520,00:45


epoch,train_loss,valid_loss,accuracy,time
0,0.168085,0.265177,0.911320,01:20
1,0.161181,0.198085,0.927240,01:20
2,0.115521,0.206109,0.929600,01:19
3,0.070494,0.238349,0.930120,01:20


In [8]:
learn.show_results()

,text,category,category_
0,"xxbos xxmaj there 's a sign on xxmaj the xxmaj lost xxmaj highway that says : \n\n * major xxup spoilers xxup ahead * \n\n ( but you already knew that , did n't you ? ) \n\n xxmaj since there 's a great deal of people that apparently did not get the point of this movie , xxmaj i 'd like to contribute my interpretation of why the plot makes perfect sense . xxmaj as others have pointed out , one single viewing of this movie is not sufficient . xxmaj if you have the xxup dvd of xxup md , you can "" cheat "" by looking at xxmaj david xxmaj lynch 's "" top 10 xxmaj hints to xxmaj unlocking xxup md "" ( but only upon second or third viewing , please . ) ;) \n\n xxmaj first of all , xxmaj mulholland xxmaj drive is",pos,pos
1,"xxbos xxmaj wrestlemania 6 , is an entertaining xxmaj wrestlemania , if not an entirely successful one . xxmaj the xxmaj ultimate xxmaj challenge , is of course worth the price of admission alone , but once again as with a lot of the early xxmaj mania 's , there 's too much filler in between . xxmaj the crowd pops for almost everything , and as always , giving us the reliable announcing team of xxunk xxmaj body . xxmaj having a xxmaj face vs xxmaj face match as the main event for a xxmaj wrestlemania , was absolutely unheard of at this time , it only made things that much more tense . \n\n xxmaj matches . \n\n xxmaj koko xxup b. xxmaj ware / w xxmaj frankie xxmaj vs "" the xxmaj model "" xxmaj rick xxmaj martel . xxmaj for a 3 or so minute match",pos,pos
2,"xxbos xxmaj this is one of four 1970s movies by xxup tv writer xxmaj lane xxmaj slate about sensationalistic murders in small towns . xxmaj they feature likable xxup tv personalities as police chiefs and quirky characters as town regulars , including light - touch love interests . xxmaj the others are : xxmaj they xxmaj only xxmaj kill xxmaj their xxmaj masters ( james xxmaj garner , 1972 ) ; and xxmaj the xxmaj girl xxmaj in xxmaj the xxmaj empty xxmaj grave and xxmaj deadly xxmaj game ( both xxmaj andy xxmaj griffith , 1977 ) . \n\n xxmaj alda 's is set near xxmaj vermont ( "" mount xxmaj angel "" next to "" horse xxmaj creek "" ) ; the others in xxmaj california , xxmaj garner 's at seaside ( "" eden xxmaj landing "" ) and xxmaj griffith 's in the mountains ( """,neg,neg
3,"xxbos "" empire xxmaj strikes xxmaj back "" director xxmaj irvin xxmaj kershner 's "" never xxmaj say xxmaj never xxmaj again , "" a remake of the 1965 xxmaj james xxmaj bond movie "" thunderball , "" does n't surpasses the xxmaj terence xxmaj young original , but this non - harry xxmaj saltzman & xxmaj albert xxup r. xxmaj broccoli film is well worth watching if you call yourself a 007 aficionado . xxmaj nevertheless , despite its shortage of clever gadgets and the lack of a vibrant musical score , "" never xxmaj say xxmaj never xxmaj again "" rates as an above - average , suspenseful doomsday thriller with top - flight performances by a seasoned cast including xxmaj sean xxmaj connery , xxmaj kim xxmaj basinger , xxmaj klaus xxmaj maria xxmaj brandauer , xxmaj max xxmaj von xxmaj sydow , xxmaj barbara xxmaj carrera",pos,pos
4,"xxbos xxmaj during an eight - year stretch of the 1980s when xxmaj charles xxmaj bronson made nine films , only one was released by a company other than the xxmaj cannon xxmaj group : ' the xxmaj evil xxmaj that xxmaj men xxmaj do , ' a tristar xxmaj films pickup from xxmaj sir xxmaj lew xxmaj grade 's xxup itc xxmaj entertainment firm . xxmaj bronson was already in the thick of his collaborations with director xxup j. xxmaj lee xxmaj thompson , which ran through numerous actioners until ' kinjite : xxmaj forbidden xxmaj subjects ' in 1989 . \n\n xxmaj expectations should run pretty high with xxmaj bronson and xxmaj thompson working for a better - funded outfit like xxup itc , but ' the xxmaj evil xxmaj that xxmaj man xxmaj do ' is a great disappointment on many levels . xxmaj while still from",neg,neg
5,"xxbos xxmaj some people drift

In [ ]:
learn.predict("That movie was wicked cool!")
# 우리 모델이 해당 모델을 positive라고 분류

('pos', tensor(1), tensor([0.1863, 0.8137]))

In [10]:
learn.predict("ieri ho piovuto tutto il giorno")

('pos', tensor(1), tensor([0.0235, 0.9765]))

In [ ]:
# 지금까지 한 것 : 위키피디아 문서로 학습한 모델을 가져와 영화 리뷰 감성 분류기로 파인튜닝
# 이제부터 IMDb 데이터셋으로 추가적인 파인튜닝
# ULMFiT은 사전훈련된 LLM (일반적인 영어 문법 구조 학습) -> label 없는 도메인 데이터로 추가학습 (fine-tuning) -> label 있는 데이터로 추가

In [11]:
dls_lm = TextDataLoaders.from_folder(path, is_lm=True, valid_pct=0.1) # 언어모델링을 위한 dataloader에서 text data를 가져온 모습
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos xxmaj chances xxmaj are uses that marvelous song by the same name throughout the film . xxmaj robert xxmaj downey , xxmaj jr . is excellent in this movie . xxmaj his extra large eyes and wonderfully variable facial expressions are part of expertise in acting as different people in diverse films . xxmaj compare xxmaj robert xxmaj downey , xxmaj jr . in xxmaj chaplin . xxmaj you will enjoy","xxmaj chances xxmaj are uses that marvelous song by the same name throughout the film . xxmaj robert xxmaj downey , xxmaj jr . is excellent in this movie . xxmaj his extra large eyes and wonderfully variable facial expressions are part of expertise in acting as different people in diverse films . xxmaj compare xxmaj robert xxmaj downey , xxmaj jr . in xxmaj chaplin . xxmaj you will enjoy xxmaj"
1,"astronauts sent to explore a newly - discovered planet must deal with several dilemmas , and they do so intelligently . xxmaj the film approaches it 's main plot theme in a unique way , and unfolds it gradually , though it can be guessed beforehand . \n\n xxmaj the acting is very good , though sometimes stiff , as some late-60s acting can be . xxmaj it can also be somewhat","sent to explore a newly - discovered planet must deal with several dilemmas , and they do so intelligently . xxmaj the film approaches it 's main plot theme in a unique way , and unfolds it gradually , though it can be guessed beforehand . \n\n xxmaj the acting is very good , though sometimes stiff , as some late-60s acting can be . xxmaj it can also be somewhat wordy"
2,"with his ill - placed attempts at humorous comments , the initially - hostile but soon softened gorgeous lady who triggers the inevitable "" unlikely "" love story , the loved ones taken hostage , and of course the careless evil adversary with his brutal minions . xxmaj everybody has seen tons of these movies already , and "" national xxmaj treasure "" is like any one of them , with only","his ill - placed attempts at humorous comments , the initially - hostile but soon softened gorgeous lady who triggers the inevitable "" unlikely "" love story , the loved ones taken hostage , and of course the careless evil adversary with his brutal minions . xxmaj everybody has seen tons of these movies already , and "" national xxmaj treasure "" is like any one of them , with only a"
3,"it is not my dislike for horror movies that makes me hate this film - i've seen such wonderful teen horrors as "" house of xxmaj wax "" , its the fact that the film leaves you with the awful understanding that by renting the video , you are supporting the creators of xxmaj within xxbos xxmaj this is a great investigation into the challenge of having a merely sexual relationship with","is not my dislike for horror movies that makes me hate this film - i've seen such wonderful teen horrors as "" house of xxmaj wax "" , its the fact that the film leaves you with the awful understanding that by renting the video , you are supporting the creators of xxmaj within xxbos xxmaj this is a great investigation into the challenge of having a merely sexual relationship with someone"
4,", this is watched by a scientist that works there on a monitor and guess what she does , well lets go in to the cage to check the stuff out , omg how dumb do those writers think human beings are come on that s the same like jumping in a fish tank with a great white shark because it ate your goldfish … pretty useless and even more xxunk i","this is watched by a scientist that works there on a monitor and guess what she does , well lets go in to the cage to check the stuff out , omg how dumb do those writers think human beings are come on that s the same like jumping in a fish tank with a great white shark because it ate your goldfish … pretty useless and even more xxunk i will"


In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], # Perplexity : exp(loss)
    path=path, wd=0.1).to_fp16()

In [13]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.384686,4.117678,0.285515,61.416447,12:58


In [14]:
learn.save('1epoch')

Path('/home/ajh1729/.fastai/data/imdb/models/1epoch.pth')

In [15]:
learn = learn.load('1epoch')

In [16]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

In [ ]:
learn.save_encoder('finetuned') # 마지막 layer만 제외하고 저장

In [ ]:
# generate random review. 문장의 다음 단어에 나올 단어를 예측하게끔 학습되었기 때문에 이런것도 가능
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75)
for _ in range(N_SENTENCES)]
print("\n".join(preds))

i liked this movie because it was very well directed . The acting was excellent , the story was never too simple , the story was something you did n't know , but the job is great . The music is beautiful as
i liked this movie because it was supposed to be a comedy . However , it was n't funny . It seemed like it was just a comedy and when it was written and directed by Richard Pryor , it was


In [ ]:
# IMDb 데이터셋 다시 load
dls_clas = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test', text_vocab=dls_lm.vocab)

In [22]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn = learn.load_encoder('finetuned')

In [ ]:
# IMDb 데이터로 파인튜닝을 해서 1epoch만 했는데도 처음 section처럼 93% 달성
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.221045,0.180914,0.930880,00:44


In [26]:
# 마지막 2개 parameter group 제외하고 모두 freeze
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time


In [27]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.182458,0.151627,0.942920,01:04


In [28]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.145205,0.174598,0.934480,01:20
1,0.119168,0.146784,0.945640,01:20


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pretrained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode a text inputs
text = "With great power comes great "
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

# Load pretrained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
'''
# 학습 모드에서 실행
model.train()
output_train = model(x)  # Dropout 활성화됨

# 추론 모드에서 실행
model.eval()
output_eval = model(x)  # Dropout 비활성화됨
'''
# model.eval() 호출 시 내부적으로 일어나는 변화
# Dropout 비활성화 : 학습 중엔 일부 뉴런을 랜덤하게 비활성화(dropout)시켜 일반화성능을 향상시키지만, 추론 모드에선 모든 뉴런을 활성화한다
# BN 값 고정 : 학습 중엔 배치별로 평균, 분산을 업데이트하지만, 추론모드에선 학습된 평균, 분산을 그대로 사용하여 안정적인 예측 수행
model.eval()

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# Get the predicted next subword
predicted_index = torch.argmax(predictions[0, -1, :]).item() # predictions = (batch_size, sequence_length, vocab_size)
# [0, -1, :] 단일문장만 입력했으므로 첫번째 배치 선택(0), 입력 문장의 마지막 단어에서 예측된 모든 단어의 확률 가져옴(-1, :), 가장 큰 확률을 가진 단어의 인덱스 반환(item())
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
print(predicted_text)

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

With great power comes great ills
